# Tutors - expected math exam results

Соревнование Kaggle https://www.kaggle.com/c/tutors-expected-math-exam-results

Участник: Aleksandr Mikhailov, https://www.kaggle.com/avidclam

In [1]:
%precision 4
import numpy as np
np.seterr(over = 'raise')
np.set_printoptions(precision=4, suppress=True)
import pandas as pd
from sklearn.model_selection import train_test_split

## Примечания

В данной работе хотелось:
- реализовать что-либо небанальное;
- получить приличный результат.

Не хотелось:
- переписывать код "из учебника".

В итоге за основу взято решающее дерево, у которого в узлах находятся модели линейной регрессии. Этот регрессор обеспечивает кусочно-линейную аппроксимацию данных. В качестве критерия деления узла выбрано улучшение коэффициента детерминации R2. Некоторые другие идеи не смогли показать должного результата (либо я не преуспел в их реализации).

## Библиотека

In [2]:
# R2 Score, обрезанный в нуле, т.е. ∈ [0,1]
def quality_score(y_true, y_pred):
    deviation = y_true - np.mean(y_true)
    residuals = y_pred - y_true
    dev_square = deviation @ deviation
    if np.isclose(dev_square, 0.0):
        score = 0.0
    else:
        score = max(1.0 - (residuals @ residuals) / dev_square, 0.0)
    return score

In [3]:
# Линейная регрессия с обязательным свободным коэффициентом
class LinReg:
    def __init__(self):
        self.A = None
    
    def _X1(self, data):
        return np.append(data, np.ones(shape=(data.shape[0], 1)), axis=1)
    
    def fit(self, X, y):        
        self.A, *_ = np.linalg.lstsq(self._X1(X), y, rcond=None)
        return self
    
    def predict(self, data):
        return self._X1(data) @ self.A

## Подготовка данных

In [4]:
TRAIN_PATH = 'train.csv'
TEST_PATH = 'test.csv'

In [5]:
class Transformer:
    target = 'mean_exam_points'
    subjects = ['physics', 'chemistry', 'biology', 'english', 'geography', 'history']
    use_features = ['age', 'years_of_experience', 'qualification', 'lesson_price']
    
    def __init__(self):
        self.subjects_encoder = None
    
    def split(self, df):
        y_df = df[self.target]
        X_df = df.drop(columns=[self.target], inplace=False)
        return train_test_split(X_df, y_df, test_size = 0.3, random_state = 2021)
    
    def fit(self, df, y):
        # школьные предметы заменим одной фичей через target encoding
        self.subjects_encoder = LinReg().fit(df[self.subjects].values, y.values)
        return self
        
    def transform(self, df):
        # Scale
        result_df = df.drop(columns=['Id'], inplace=False).copy()
        # Add subject feature
        subject_feature = self.subjects_encoder.predict(df[self.subjects].values)
        return np.hstack([result_df[self.use_features].values, subject_feature[:, np.newaxis]])
    
    def fit_transform(self, df, test_df=None):
        if test_df is None:  # split df
            train_df, test_df, y_train, y_test = self.split(df)
        else:  # работаем с предоставленными тестовыми данными без y
            y_train = df[self.target]
            y_test = None
            train_df = df.drop(columns=[self.target], inplace=False)
        y_test_values = y_test.values if y_test is not None else None
        self.fit(train_df, y_train)
        X_train, X_test = self.transform(train_df), self.transform(test_df)
        return X_train, X_test, y_train.values, y_test_values

In [6]:
# Функция моделирования и вывода результатов на training set
def predict_show(model):
    raw_train_df = pd.read_csv(TRAIN_PATH)
    X_train, X_test, y_train, y_test = Transformer().fit_transform(raw_train_df)
    model.fit(X_train, y_train)
    train_pred = model.predict(X_train)
    train_score = quality_score(y_train, train_pred)
    test_pred = model.predict(X_test)
    test_score = quality_score(y_test, test_pred)
    return f"TrainR2={train_score:.3f}, TestR2={test_score:.3f}"

In [7]:
# Функция моделирования и сохранения результатов для соревнования
def predict_save(model, outfile):
    raw_train_df = pd.read_csv(TRAIN_PATH)
    raw_test_df = pd.read_csv(TEST_PATH)
    X_train, X_test, y_train, y_test = Transformer().fit_transform(raw_train_df, raw_test_df)
    prediction = model.fit(X_train, y_train).predict(X_test)
    prediction = np.array(prediction + 0.5, dtype=int)  # предсказываем целые баллы
    result_df = pd.DataFrame({'Id': raw_test_df['Id'], 'mean_exam_points': prediction})
    result_df.to_csv(outfile, index=None)

## Базовая метрика

Сравнивать качество моделей будем с результатами линейной регрессии.

In [8]:
predict_show(LinReg())

'TrainR2=0.651, TestR2=0.644'

In [9]:
predict_save(LinReg(), 'submission_linreg.csv')

## Рабочая модель

In [10]:
class LinRegTree:    
    def __init__(self, min_samples_leaf=100, min_samples_predict=20):
        self.kwargs = {k:v for k, v in locals().items() if k != 'self'}
        self.msl = min_samples_leaf
        self.msp = min_samples_predict
        self.split_feature = None  # splitting feature index
        self.t = None  # splitting threshold
        self.reg, self.right, self.left = LinReg(), None, None
    
    def fit_only(self, X, y):        
        self.reg.fit(X, y)
        return self
    
    def fit(self, X, y):
        self.fit_only(X, y)
        self.split(X, y)
        return self
    
    def split(self, X, y):
        n_samples, n_features = X.shape
        y_pred = self.reg.predict(X)
        best_score = quality_score(y, y_pred)
        rmod, lmod = LinRegTree(**self.kwargs), LinRegTree(**self.kwargs)
        for index in range(n_features):
            for t in np.unique(X[:, index]):                
                rmask = X[:, index] > t
                lmask = ~rmask
                if self.msl <= np.sum(rmask) <= n_samples - self.msl:
                    y_pred_r = rmod.fit_only(X[rmask], y[rmask]).predict(X[rmask])
                    y_pred_l = lmod.fit_only(X[lmask], y[lmask]).predict(X[lmask])
                    split_score = quality_score(np.append(y[rmask], y[lmask]),
                                                np.append(y_pred_r, y_pred_l))
                    if split_score > best_score:
                        best_score, self.split_feature, self.t = split_score, index, t
                        self.right, self.left = rmod, lmod
                        X_right, X_left = X[rmask], X[lmask]
                        y_right, y_left = y[rmask], y[lmask]
                        rmod, lmod = LinRegTree(**self.kwargs), LinRegTree(**self.kwargs)
        if not (self.right is None or self.left is None):
            self.right.split(X_right, y_right)
            self.left.split(X_left, y_left)
        return self
    
    def predict(self, data):
        n_samples, _ = data.shape
        y_pred = np.zeros(n_samples)
        split_predict = False
        if self.split_feature:            
            rmask = data[:, self.split_feature] > self.t
            if self.msp <= np.sum(rmask) <= n_samples - self.msp:
                split_predict = True
                lmask = ~rmask                
                y_pred[rmask] = self.right.predict(data[rmask])
                y_pred[lmask] = self.left.predict(data[lmask])
        return y_pred if split_predict else self.reg.predict(data) 

In [11]:
lrt_model = LinRegTree(min_samples_leaf=40, min_samples_predict=40)
predict_show(lrt_model)

'TrainR2=0.805, TestR2=0.779'

In [12]:
predict_save(lrt_model, 'submission_linregtree.csv')

Получился довольно высокий результат, который говорит о том, что данные неплохо приближаются кусочной линейной регрессией. Дальнейшее улучшение результата возможно, но нужно понимать, что за каждым улучшением качества предсказания во втором-третьем знаке будет стоять существенное увеличение сложности модели и потребность в вычислительных ресурсах.

## Потенциально улучшенная модель

Традиционно за моделью решающего дерева необходимо опробовать лес. Будем строить модели на основе бутстрап-выборок и отбирать лучшие по оценке качества на Out-of-bag данных. Эта же оценка будет использоваться в качестве веса модели.

In [13]:
class LinRegRF:
    def __init__(self, min_samples_leaf=100, min_samples_predict=20, n_estimators=10, n_estimators_keep=5, rng=None):
        self.msl = min_samples_leaf
        self.msp = min_samples_predict
        self.n_estimators = n_estimators
        self.n_estimators_keep = n_estimators_keep
        if rng is None or isinstance(rng, int):
            rng = np.random.default_rng(rng)
        self.rng = rng
        self.estimators = []
        self.oob_scores = []
    
    def fit(self, X, y):
        n_samples, _ = X.shape
        for _ in range(self.n_estimators):
            bootstrap_idx = self.rng.integers(0, n_samples, size = n_samples)
            oob_idx = np.setdiff1d(np.arange(n_samples), bootstrap_idx)            
            estimator = LinRegTree(min_samples_leaf=self.msl, min_samples_predict=self.msp)
            estimator.fit(X[bootstrap_idx], y[bootstrap_idx])
            oob_pred = estimator.predict(X[oob_idx])
            oob_score = quality_score(y[oob_idx], oob_pred)
            self.estimators.append(estimator)
            self.oob_scores.append(oob_score)
        return self
    
    def predict(self, data):
        oob_scores_all = np.array(self.oob_scores)
        keep_idx = np.argsort(-oob_scores_all)[:self.n_estimators_keep]  # Top N
        oob_scores = oob_scores_all[keep_idx]
        weighted_scores = oob_scores / np.sum(oob_scores)
        predictions = [self.estimators[i].predict(data) for i in keep_idx]
        return weighted_scores @ np.vstack(predictions)

In [14]:
rng = np.random.default_rng(2021)
lrrf_model = LinRegRF(min_samples_leaf=120, min_samples_predict=40, n_estimators=17, n_estimators_keep=7)
predict_show(lrrf_model)

'TrainR2=0.797, TestR2=0.780'

In [15]:
predict_save(lrrf_model, 'submission_linregrandomforest.csv')

In [16]:
# Длинные вычисления
long_model = LinRegRF(min_samples_leaf=100, min_samples_predict=40, n_estimators=470, n_estimators_keep=47, rng=rng)
predict_show(long_model)

'TrainR2=0.798, TestR2=0.783'

In [17]:
predict_save(long_model, 'submission_linregrandomforest_long.csv')

## Результаты Kaggle

- результат линейной (базовой) модели: 0.65821
- результат работы модели LinRegTree: 0.78157
- лучший результат работы модели на основе бутстрап-выборок: 0.78656 (типичным следует назвать 0.785)

**Вывод**: модель дерева решений с встроенными в листья линейными регрессорами обладает приемлемой скоростью работы и дает высокий результат при работе на данных, хорошо отвечающих на кусочно-линейную аппроксимацию. Ансамблевые модели дают незначительное улучшение качества модели при значительном потреблении вычислительных ресурсов.